In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib qt

In [2]:
from ICARUS.Aerodynamics.Potential.wing_lspt import Wing_LSPT
from ICARUS.Aerodynamics.Potential.vorticity import symm_wing_panels

from ICARUS.Database import DB as db
from ICARUS.Vehicle.plane import Airplane

import numpy as np
import pandas as pd


# Import Plane

In [5]:
from Vehicles.Planes.e190_cruise import e190_cruise
from Vehicles.Planes.e190_takeoff import e190_takeoff_generator

# embraer_cruise: Airplane = e190_cruise(name="e190_cruise_3")
embraer_to: Airplane = e190_takeoff_generator(name="e190_takeoff_3")
# from examples.Planes.simple_wing import airplane
# plane: Airplane = airplane
embraer_to.visualize()
plane: Airplane = embraer_to

# Import and setup Environment

In [6]:
from ICARUS.Environment.definition import EARTH_ISA

ALTITUDE: dict[str, int] = {
    'e190_cruise_3': 12000,
    'e190_takeoff_3': 0,
    'bmark': 0
}

# OUR ATMOSPHERIC MODEL IS NOT COMPLETE TO HANDLE TEMPERATURE VS ALTITUDE
TEMPERATURE: dict[str, int] = {
    'e190_cruise_3': 273 - 50,
    'e190_takeoff_3': 273 + 15,
    'bmark': 273+15
}

EARTH_ISA._set_pressure_from_altitude_and_temperature(ALTITUDE[plane.name],  TEMPERATURE[plane.name])

# Create Simulation Object And set parameters

In [8]:
UMAG: dict[str, float] = {
    'e190_cruise_3': 232,
    'e190_takeoff_3':20,
    'bmark': 20
}

plane_pot = Wing_LSPT(
    plane=plane,
    environment= EARTH_ISA,
    alpha= 0,
    beta= 0, 
)


In [9]:
plane_pot.plot_grid(show_wake=False)

# Run Simulation

In [10]:
angles = np.linspace(-6, 10, 2*(10-(-6))+1)
df: pd.DataFrame = plane_pot.aseq(
    angles=angles,
    umag = UMAG[plane.name],
    solver_fun= symm_wing_panels,
)


Solving for a and b
	Could not interpolate polars! Got error:
	Reynolds 7558878.474675898 not in database! Min Reynolds is 18430000.0
	Could not interpolate polars! Got error:
	Reynolds 7558876.799184084 not in database! Min Reynolds is 18430000.0
	Could not interpolate polars! Got error:
	Reynolds 7558876.837926163 not in database! Min Reynolds is 18430000.0
	Could not interpolate polars! Got error:
	Reynolds 7558878.590890306 not in database! Min Reynolds is 18430000.0
	Could not interpolate polars! Got error:
	Reynolds 7558882.0575413555 not in database! Min Reynolds is 18430000.0
	Could not interpolate polars! Got error:
	Reynolds 7558887.236820976 not in database! Min Reynolds is 18430000.0
	Could not interpolate polars! Got error:
	Reynolds 7558894.127147988 not in database! Min Reynolds is 18430000.0
	Could not interpolate polars! Got error:
	Reynolds 7558902.726418861 not in database! Min Reynolds is 18430000.0
	Could not interpolate polars! Got error:
	Reynolds 7558913.0320083

# Plot Polars

In [ ]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots(2,2)

ax[1,0].plot(df['AoA'], df['CL'],label = "Potential")
ax[1,0].plot(df['AoA'], df['CL_2D'], label = "2D")
ax[1,0].set_xlabel("AoA")
ax[1,0].set_ylabel("CL")

ax[0,1].plot(df['AoA'], df['CD'],label = "Potential")
ax[0,1].plot(df['AoA'], df['CD_2D'], label = "2D")
ax[0,1].set_xlabel("AoA")
ax[0,1].set_ylabel("CD")

ax[0,0].plot(df['AoA'], df['Cm'],label = "Potential")
ax[0,0].plot(df['AoA'], df['Cm_2D'], label = "2D")
ax[0,0].set_xlabel("AoA")
ax[0,0].set_ylabel("Cm")

ax[1,1].plot(df['CD'], df['CL'],label = "Potential")
ax[1,1].plot(df['CD_2D'], df['CL_2D'], label = "2D")
ax[1,1].set_xlabel("CD")
ax[1,1].set_ylabel("CL")

for a in ax.flatten():
    a.grid()
    a.minorticks_on()
    a.axvline(0, color="black", linestyle="-")
    a.axhline(0, color="black", linestyle="-")
ax[0,0].legend() 

plt.show()

# Save Results to DB

In [ ]:
from ICARUS.Aerodynamics.Potential.lifting_line import save_results

save_results(
    db =db,
    plane = plane,
    df = df,
)

# RUN A SPECIFIC CASE TO GET MORE RESULTS

In [18]:
plane_pot.alpha = 1*np.pi / 180
Uinf = UMAG[plane.name] * np.cos(plane_pot.alpha) * np.cos(plane_pot.beta)
Vinf = UMAG[plane.name] * np.cos(plane_pot.alpha) * np.sin(plane_pot.beta)
Winf = UMAG[plane.name] * np.sin(plane_pot.alpha) * np.cos(plane_pot.beta)
Q = np.array((Uinf, Vinf, Winf))

plane_pot.solve_wing_panels(Q,symm_wing_panels)
plane_pot.get_gamma_distribution()
plane_pot.get_gamma_distribution()
plane_pot.get_strip_reynolds(UMAG[plane.name])
plane_pot.get_aerodynamic_loads(UMAG[plane.name])
plane_pot.calculate_strip_induced_velocities()
plane_pot.calculate_strip_gamma()


Using previous solution for a and b! Be smart
	Could not interpolate polars! Got error:
	Reynolds 3840789.40673543 not in database! Min Reynolds is 4495000.0
- Angle 1.0
	--Using no penetration condition:
		L:19982.660794256233	|	D (Trefftz Plane):863.0024024801555	D2:762.6456278997453	|	My:23407.983667594683
		CL:0.8200579494083021	|	CD_ind:0.035416303554316325	|	Cm:0.3755285544936944


In [21]:
# plane_pot.plot_w_induced_strips(umag=UMAG[plane.name])
# plane_pot.plot_gamma_distribution()
plane_pot.plot_lift_drag_strips()
plane_pot.plot_lift_drag_panels(umag= UMAG[plane.name])

In [23]:
plane_pot.strip_reynolds
import matplotlib.pyplot as plt

In [24]:
plt.plot(plane_pot.strip_reynolds
         )